# **Tutorial:** Decision Trees for Classification

Decision trees are a supervised learning method used for classification (the focus of this tutorial) and regression. The goal is to create a model that predicts the value of a label $y_i$ by learning simple decision rules (if-else rules) inferred from the features vector $x_i$.

**Some advantages of decision trees are:**

- Trees can be visualised, thus they're simple to interpret and explain. 
- Require little data preparation. For instance, other techniques often require data normalisation.
- Very fast. The cost of using the tree is logarithmic in the number of data points used to train the tree.

**Some disadvantages of decision trees are:**

- Decision tree learners can create over-complex trees that do not generalise the data well. 
- Decision trees can be unstable, small variations in the data might result in a completely different tree being generated.
- Decision tree learners create biased trees if some classes dominate. It is therefore recommended to balance the dataset prior to fitting with the decision tree.


## Structure

Three kinds of nodes:
- **Root Node**: no parent node, question giving rise to two children nodes.
- **Internal Node**: one parent node, question giving rise to two children nodes.
- **Leaf Node**: one parent node, no children nodes (shows the prediction).
- **Branch**: connection between parent and child node, signifies true or false.  

<img src="images/tree.PNG" width="800">

In the tree example above we see a max depth of 2 (root node counting as a depth of 0), with a total of 2 internal nodes, and 4 leaves. The values A, B, C are considered split points for the different features of $x_i$.

## Tree-Generation Algorithms

Decision tree learning is the construction of a decision tree from class-labeled training tuples. A decision tree is a flow-chart-like structure, where each internal (non-leaf) node denotes a test on a feature, each branch represents the outcome of a test, and each leaf node holds a class label. The topmost node in a tree is the root node. There are many specific decision-tree algorithms. Notable ones include: 

**ID3** (Iterative Dichotomiser 3) was developed in 1986 by Ross Quinlan. The algorithm creates a multiway tree, finding for each node (i.e. in a greedy manner) the categorical feature that will yield the largest information gain for categorical targets. Trees are grown to their maximum size and then a pruning step is usually applied to improve the ability of the tree to generalise to unseen data.

**C4.5** is the successor to ID3 and removed the restriction that features must be categorical by dynamically defining a discrete attribute (based on numerical variables) that partitions the continuous attribute value into a discrete set of intervals. C4.5 converts the trained trees (i.e. the output of the ID3 algorithm) into sets of if-then rules. These accuracy of each rule is then evaluated to determine the order in which they should be applied. Pruning is done by removing a rule’s precondition if the accuracy of the rule improves without it.

**C5.0** is Quinlan’s latest version release under a proprietary license. It uses less memory and builds smaller rulesets than C4.5 while being more accurate.

**CHAID** (CHi-squared Automatic Interaction Detector) is based on adjusted significance testing and performs multi-level splits when computing classification trees

**MARS** (Multivariate Adaptive Regression Splines): extends decision trees to handle numerical data better (regression only).

**CART** (Classification and Regression Trees) is very similar to C4.5, but it differs in that it supports numerical target variables (regression) and does not compute rule sets. CART constructs binary trees using the feature and threshold that yield the largest information gain at each node.

- scikit-learn uses an optimised version of the CART algorithm; however, scikit-learn implementation does not support categorical features for now.



## Mathematical Formulation

Most of the aforementioned tree generating algorithms are based on the following mathematical formulation. We are given a set of training samples in the form of $(x_i, y_i)$ where $x_i$ is the predictor variable with $k$ features (attributes) $x_i = x_{i,1},x_{i,2},...,x_{i,k}$, and $y_i$ is the target variable (label). A decision tree recursively partitions (divides) the space such that the samples with the same labels are grouped together.

---
*Recursion*: 
- the method of solving a problem where the solution depends on solutions to smaller instances of the same problem (as opposed to iteration).
- the process a procedure goes through when one of the steps of the procedure involves invoking the procedure itself. 
- a procedure that goes through recursion is said to be 'recursive'. 
---

The nodes of a classification tree are grown recursively. In other words, the creation of an internal node or leaf depends on the state of its predecessors. To produce the best leafs possible, at each node, a tree asks a question involving one feature and a split point. But how does it know which feature, and which split point to pick? It does so by considering that every node contains information, and the tree's goal is to maximize of the information present at that node. Algorithms for constructing decision trees usually work top-down, by choosing a variable at each step that best splits the set of items.

First we gather all the data at node $m$, represented by $Q_m$ with a total of $N_m$ number of data samples. We then create a set $\theta^{m}$ of candidate splits $\theta = (f,t_m)$ consisting of a feature $f$ and threshold $t_m$ for that feature at node $m$. From these candidates we partition the data into $Q^{left}_{m}(\theta)$ and $Q^{right}_{m}(\theta)$ subsets, each with a total number of samples of $n_{left}$ and $n_{right}$.

Next we define the information gain $G(Q_m, \theta)$ at node $m$, for each candidate split, using an impurity function $H()$. Information gain measures how much (useful) information a feature gives us about the class. Features that perfectly partition the data by a class give maximal information. Unrelated features to a class should give no information. Information gain is calculated by

$$
G(Q_m, \theta) = \frac{n_{left}}{N_m} H(Q^{left}_{m}(\theta)) + \frac{n_{right}}{N_m} H(Q^{right}_{m}(\theta))
$$

The best parameters are selected by maximizing information gain

$$
\theta^* = \text{argmax}_{\theta \in \theta^{m}} G(Q_m, \theta) 
$$

We then recurse for the subsets $Q_{left}(\theta^*)$ and $Q_{right}(\theta^*)$ until the max depth for the tree is reached, $N_m$ is less then the minimum samples allowed per node, $N_m =  1$, or the information gain of that node is 0 in which case it becomes a leaf.

### Impurity Measures (Classification Criteria)

If a target is a classification outcome taking on values $1,2,...,K$ for node $m$, representing a region $R_m$ with $N_m$ observations, let

$$
\hat{p}_{mk} = \frac{1}{N_m} \sum_{x_i \in R_m} I(y_i = k)
$$

be the proportion of class $k$ observations in node $m$, where $I()$ is an indicator function. From this proportion value we can define two main measures $H()$ for impurity.

**Gini Index**: $ H(Q_m) = \sum_{k=1}^{K} \hat{p}_{mk}(1-\hat{p}_{mk}) $

**Entropy**: $ H(Q_m) = -\sum_{k=1}^{K} \hat{p}_{mk} \log \hat{p}_{mk} $

Gini is intended for continuous features and is faster to compute, but it does not perform well when the number of classes is large (ideal for binary classification).

Entropy is used more for categorical features, is biased toward multivalued features, and used more for exploratory data analysis. Gini is used by CART, while entropy is used by ID3, C4.5, and C5.0 tree-generation algorithms.


## Sklearn Code Tutorial

In [ ]:
# Import DecisionTreeClassifier from sklearn.tree
from sklearn.tree import DecisionTreeClassifier

# Instantiate a DecisionTreeClassifier 'dt' 
dt = DecisionTreeClassifier()

# Fit dt to the training set
dt.fit(X_train, y_train)

# Predict test set labels
y_pred = dt.predict(X_test)

# Import accuracy_score
from sklearn.metrics import accuracy_score

# Predict test set labels
y_pred = dt.predict(X_test)

# Compute test set accuracy  
acc = accuracy_score(y_pred, y_test)
print("Test set accuracy: {:.2f}".format(acc))

## Sklearn DecisionTreeClassifier Main Hyperparameters

From scikit-learn v0.20.2.

**criterion**: The impurity measure used.
- string, default = 'gini' 
- 'gini' = Gini Index $\rightarrow$ ideal for binary classification
- 'entropy' = Entropy
    
**max_depth**: The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples. If too high it overfits training data, if too low it underfits training data. The best parameter is to grid/random search it with cross-validation
- int, default = None
- grid search 2 to 100, include option None in search

**min_samples_split**: The minimum number of samples required to split an internal node. Too high and it may underfit.
- int, float, default = 2
- If int, then consider min_samples_split as the minimum number.
- If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.
- do grid search with a float number 0.1 to 0.5

**min_samples_leaf**: The minimum number of samples required to be at a leaf node. 
- int, float, default = 1
- If int, then consider min_samples_leaf as the minimum number.
- If float, then min_samples_leaf is a fraction and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node.
- A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.
- A smaller leaf makes the model more prone to capturing noise in train data. 
- do grid search with a float number 0.1 to 0.5

**max_features**: The number of features to consider when looking for the best split. This mechanism is used to control overfitting. But too high and it may underfit.
- int, float, string or None, default=None
- If int, then consider max_features features at each split.
- If float, then max_features is a fraction and int(max_features * n_features) features are considered at each split.
- If “auto”, then max_features=sqrt(n_features).
- If “sqrt”, then max_features=sqrt(n_features).
- If “log2”, then max_features=log2(n_features).
- If None, then max_features=n_features.
- Unless n_features is huge, this may not affect performance. Grid search 1 to n_features or ['auto', 'sqrt', 'log2', 'None'].

**max_leaf_nodes**: Limit number of leafs created. If None then unlimited number of leaf nodes.
- int or None, default=None
- Grid search = [5, 25, 50, 100, 250, 500, 'None']
- don't really need to tune if tuning min_samples_split and min_samples_leaf


